In [1]:
import os
import json
from pathlib import Path
from pprint import pprint
from backend.easyocr import EasyOCR
from backend.text_evaluator import extract_product_data
from backend.recommender import recommend_cheaper
from backend.donut import DonutOCR

d:\miniconda\envs\final_proj\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
d:\miniconda\envs\final_proj\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.ai.generativelanguage_v1beta once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.ai.generativelanguage_v1beta past that date.
  warnings.warn(message, FutureWarning)
d:\miniconda\envs\final_proj\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

In [ ]:
# -------------------------
# Configuration
# -------------------------
# Base dataset folder
SCREENSHOT_DIR = Path(r"D:\personalProjects\final_proj\src\dataset")

# Get all screenshots in the dataset 
screenshots = list(SCREENSHOT_DIR.rglob("*.png"))

# OCR engines
ocr_engines = ["easyocr", "donut"]

# EasyOCR filtering options
easyocr_filters = [True, False]  # True = filtered, False = raw

# Search engines
search_engines = ["serpapi", "ddg"]

# -------------------------
# INITIALIZE OCR
# -------------------------
easyocr_engine = EasyOCR(languages=["en", "id"])
donut_engine = DonutOCR()

Using CPU. Note: This module is much faster with a GPU.
Loading weights: 100%|██████████| 484/484 [00:00<00:00, 521.54it/s, Materializing param=encoder.encoder.layers.3.blocks.1.output.dense.weight]                         
The tied weights mapping and config for this model specifies to tie decoder.model.decoder.embed_tokens.weight to decoder.lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [7]:
# -------------------------
# EXPERIMENT LOOP
# -------------------------
results = []

for img_path in screenshots:
    for ocr in ocr_engines:
        for search in search_engines:
            # Handle EasyOCR filtering separately
            if ocr == "easyocr":
                for filter_option in easyocr_filters:
                    print(f"Running: {img_path.name} | OCR={ocr} | Filter={filter_option} | Search={search}")

                    # OCR step
                    if filter_option:
                        ocr_boxes = easyocr_engine.get_relevant_boxes_mobile(img_path) \
                            if "mobile" in img_path.name.lower() else easyocr_engine.get_relevant_boxes_pc(img_path)
                        ocr_text = "\n".join([b["text"] for b in ocr_boxes])
                    else:
                        ocr_text = easyocr_engine.run(img_path)["text"]

                    # Product extraction
                    try:
                        product_data = extract_product_data(ocr_text)
                    except Exception as e:
                        product_data = {"error": str(e)}

                    # Recommender
                    try:
                        cheaper_products = recommend_cheaper(product_data)
                    except Exception as e:
                        cheaper_products = {"error": str(e)}

                    results.append({
                        "screenshot": img_path.name,
                        "ocr": "EasyOCR",
                        "filter": filter_option,
                        "search_engine": search,
                        "ocr_text": ocr_text,
                        "product_data": product_data,
                        "cheaper_products": cheaper_products
                    })

            elif ocr == "donut":
                print(f"Running: {img_path.name} | OCR={ocr} | Search={search}")

                # Donut OCR
                try:
                    ocr_output = donut_engine.run(img_path)
                    ocr_text = json.dumps(ocr_output["parsed"])
                except Exception as e:
                    ocr_text = str(e)

                # Product extraction
                try:
                    product_data = extract_product_data(ocr_text)
                except Exception as e:
                    product_data = {"error": str(e)}

                # Recommender
                try:
                    cheaper_products = recommend_cheaper(product_data)
                except Exception as e:
                    cheaper_products = {"error": str(e)}

                results.append({
                    "screenshot": img_path.name,
                    "ocr": "DonutOCR",
                    "filter": None,
                    "search_engine": search,
                    "ocr_text": ocr_text,
                    "product_data": product_data,
                    "cheaper_products": cheaper_products
                })

Running: lazada_1.png | OCR=easyocr | Filter=True | Search=serpapi


d:\miniconda\envs\final_proj\lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


[{'type': 'text', 'text': '[\n  {\n    "name": "Harga 3 Pcs Oli Motor Matic Mpx2 800Ml 10W-30 Khusus Honda Matic Vario Beat Scoopy",\n    "price_idr": 25097,\n    "store": "Lazada Indonesia",\n    "product_url": null\n  },\n  {\n    "name": "Oli MPX2 800 ml AHM Oil Matic MPX 2 0.8L",\n    "price_idr": 29000,\n    "store": "Shopee - Indonesia",\n    "product_url": null\n  },\n  {\n    "name": "OLI MPX2 0.8L Oli motor matic honda Mpx2 0.8 Oli matic Mpx 2 0.8L Oli mpx2 matic 800ml oli mpx",\n    "price_idr": 29900,\n    "store": "Shopee - Indonesia",\n    "product_url": null\n  }\n]', 'extras': {'signature': 'CuclAb4+9vstj3k3NIQaKdatpoRzbi6A7KV5u5TUnEdTC8r31mdTbFL9R8BOW0LB1l8ltD9E16bZFq4aHfOWm1c2jHKGbam8DITif3GUfOa+sRphYz2KEt435gTH+sJE9bSthDgrdWIRC+ty+h9/daZMOpjrpIEgqC5BkTU6Vg35i3HEF5vZuFLI5oJ3rMxptIEr3UL9awcHcYw7h/sSV4ZrQcwFf7B/aKSucqVKvMTUeEFXJoRlZlmFMYJqI72wZPqEMTd3XVYFFiDPGrjuemJVFW+xpwOIjjkxBaI3CHzWNYGCPOHUAZQ9JX0Rt/MjVIzmoHOMpkhSb8dCdOJdE25y+JCdi7hAOLyGLUuOveSOmdqafdcMeue2COb6equYZY

d:\miniconda\envs\final_proj\lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


[{'type': 'text', 'text': '[\n  {\n    "name": "Oli Motor Yamalube Matic 0.8 Liter (800 ml)",\n    "price_idr": 30000,\n    "store": "Shopee - Indonesia",\n    "product_url": "https://shopee.co.id/search?keyword=Oli%20Motor%20Yamalube%20Matic%200.8%20Liter"\n  },\n  {\n    "name": "Oil Oli Mesin Matic 4T 10W-40 0,8ML ECCO",\n    "price_idr": 30240,\n    "store": "Shopee - Indonesia",\n    "product_url": "https://shopee.co.id/search?keyword=Oil%20Oli%20Mesin%20Matic%204T%2010W-40%200,8ML%20ECCO"\n  },\n  {\n    "name": "OLI MOTOR MATIC FULL SYNTHETIC 800ml 0.8L kualitas bagus lebih melindungi mesin",\n    "price_idr": 31500,\n    "store": "Shopee - Indonesia",\n    "product_url": "https://shopee.co.id/search?keyword=OLI%20MOTOR%20MATIC%20FULL%20SYNTHETIC%20800ml%200.8L"\n  }\n]', 'extras': {'signature': 'CuNLAb4+9vsCRsMJwoYaFe9LNxhcfIuCzohnZWssiPHxBFXtN4ujj0dEZFzDsTfgu7RVJnptmzc3euNquVFLJ+0Flityqmlq2+9jrX+oRY3uSbZdDd09NCnEngRTpTWt4B5z8MWCkyBKUK1fXi4tkaQBHzgtDZQFhUuVXKlR4Z6ga2AIHf8kEmb8l

...

[
  {
    "name": "Pedigree 130 Gr Makanan Anjing Basah Puppy Chicken In Gravy Pouch",
    "price_idr": 9900,
    "store": "ruparupa",
    "product_url": "https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcTDHK778LZYCYT_Ei6tSJMe8sZsqhBn3nvar4fDCNCb4BLtqF2TzGF4L1RrwZyN8hfukbZTgEd-LOIsr6W4XA7kmUSGccw8nocHTD6YZCVZKxbaORc_wy_qp"
  },
  {
    "name": "Makanan Anjing Basah PawPaw Pate Puppy 400gr From Turki Eropa - Dog Wet Food - Beef",
    "price_idr": 19277,
    "store": "tokopedia.com",
    "product_url": "https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQ-DQINTAczTO_ZY-1DqYdCP5ch3GgfaVCG2faC-4LcjyVnGO9mlrP4rv-YXHCXnhVsOTZCH44zVp3xiZpRO0xRoRxotpzz_WUmF4rDe2gYcfkO6u6dH7M34A"
  },
  {
    "name": "Pedigree Puppy Pouch 80 gr – Makanan Basah Anjing dengan Nutrisi Lengkap untuk Tumbuh Kembang Sehat",
    "price_idr": 8000,
    "store": "Petshop Indonesia",
    "product_url": "https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRQqf_1Emx-Para5Hp4la7Voo_0r6Xq1JRycmrFHEQt6JnXj9bGQ

d:\miniconda\envs\final_proj\lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


[
  {
    "name": "Pedigree 130 Gr Makanan Anjing Basah Puppy Chicken In Gravy Pouch",
    "price_idr": 9900,
    "store": "ruparupa",
    "product_url": "https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcTDHK778LZYCYT_Ei6tSJMe8sZsqhBn3nvar4fDCNCb4BLtqF2TzGF4L1RrwZyN8hfukbZTgEd-LOIsr6W4XA7kmUSGccw8nocHTD6YZCVZKxbaORc_wy_qp"
  },
  {
    "name": "Makanan Anjing Basah PawPaw Pate Puppy 400gr From Turki Eropa - Dog Wet Food - Beef",
    "price_idr": 19277,
    "store": "tokopedia.com",
    "product_url": "https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQ-DQINTAczTO_ZY-1DqYdCP5ch3GgfaVCG2faC-4LcjyVnGO9mlrP4rv-YXHCXnhVsOTZCH44zVp3xiZpRO0xRoRxotpzz_WUmF4rDe2gYcfkO6u6dH7M34A"
  },
  {
    "name": "Pedigree Puppy Pouch 80 gr – Makanan Basah Anjing dengan Nutrisi Lengkap untuk Tumbuh Kembang Sehat",
    "price_idr": 8000,
    "store": "Petshop Indonesia",
    "product_url": "https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRQqf_1Emx-Para5Hp4la7Voo_0r6Xq1JRycmrFHEQt6JnXj9bGQ

d:\miniconda\envs\final_proj\lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


[
  {
    "name": "Pedigree 130 Gr Makanan Anjing Basah Puppy Chicken In Gravy Pouch",
    "price_idr": 9900,
    "store": "ruparupa",
    "product_url": "https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcTDHK778LZYCYT_Ei6tSJMe8sZsqhBn3nvar4fDCNCb4BLtqF2TzGF4L1RrwZyN8hfukbZTgEd-LOIsr6W4XA7kmUSGccw8nocHTD6YZCVZKxbaORc_wy_qp"
  },
  {
    "name": "Makanan Anjing Basah PawPaw Pate Puppy 400gr From Turki Eropa - Dog Wet Food - Beef",
    "price_idr": 19277,
    "store": "tokopedia.com",
    "product_url": "https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQ-DQINTAczTO_ZY-1DqYdCP5ch3GgfaVCG2faC-4LcjyVnGO9mlrP4rv-YXHCXnhVsOTZCH44zVp3xiZpRO0xRoRxotpzz_WUmF4rDe2gYcfkO6u6dH7M34A"
  },
  {
    "name": "Pedigree Puppy Pouch 80 gr – Makanan Basah Anjing dengan Nutrisi Lengkap untuk Tumbuh Kembang Sehat",
    "price_idr": 8000,
    "store": "Petshop Indonesia",
    "product_url": "https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRQqf_1Emx-Para5Hp4la7Voo_0r6Xq1JRycmrFHEQt6JnXj9bGQ

In [ ]:
# -------------------------
# SAVE RESULTS
# -------------------------
with open("experiment_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"All experiments done! Total runs: {len(results)}")

All experiments done! Total runs: 36


In [9]:
# Clean the signatures 

import json

def remove_keys(obj, keys_to_remove):
    """
    Recursively remove specified keys from JSON-like object.
    """
    if isinstance(obj, dict):
        return {
            key: remove_keys(value, keys_to_remove)
            for key, value in obj.items()
            if key not in keys_to_remove
        }
    elif isinstance(obj, list):
        return [remove_keys(item, keys_to_remove) for item in obj]
    else:
        return obj

input_file = "experiment_results.json"   
output_file = "experiment_results_cleaned.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ====== REMOVE KEYS ======
keys_to_remove = {"extras", "signatures"}
cleaned_data = remove_keys(data, keys_to_remove)

# ====== SAVE CLEANED JSON ======
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=4)

